In [15]:
import numpy as np
import re
from random import sample
import tensorflow as tf
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM, Embedding
from tensorflow.keras import layers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as K
from feature_funcs import *
from keras.utils import to_categorical

In [16]:
import os
os.chdir("../data")

### Prepare Data

In [17]:
with open("jig_train.txt") as my_file:
    abc_text = my_file.read()

In [ ]:
num_to_char, char_to_num = create_dictionaries(abc_text)
vocab_length = len(num_to_char)

text_nums_train = encoder(abc_text, char_to_num)

x_vals, y_vals = create_training3(text_nums_train, 200, vocab_length)

### Build RNN

Sources: <br />
https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5  <br />
https://github.com/aamini/introtodeeplearning/blob/master/lab1/Part2_Music_Generation.ipynb  <br /> https://medium.com/datadriveninvestor/music-generation-using-deep-learning-85010fb982e2 
<br /> https://keras.io/examples/lstm_text_generation/

In [ ]:
model = tf.keras.Sequential()
# model.add(layers.Embedding(vocab_length, 256, batch_input_shape=x_vals.shape))
model.add(layers.LSTM(128, input_shape=(x.shape[1], x.shape[2]), return_sequences=True))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.LSTM(512))
model.add(layers.Dense(vocab_length, activation='softmax'))

model.summary()


In [ ]:
# rmsprop: Divide the learning rate for a weight by a running average of the
# magnitues of the recent gradients for that weight
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
# Fit model
model.fit(x, y, epochs=20, batch_size=50)

In [ ]:
predictions = model.predict(x)
predictions

In [ ]:
text_predictions = decoder(predictions, num_to_char)
print(text_predictions)